In [7]:
import soccerdata as sd 
import pandas as pd
import os
import numpy as np
#from cache_pandas import cache_to_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
#from tpot import TPOTClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [99]:
def home_away(df):
    c1 = df.filter(like='HomeTeam').columns

    c2 = c1.str.replace('HomeTeam', 'Team')
    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    c1 = df.filter(like='AwayTeam').columns

    c2 = c1.str.replace('AwayTeam', 'Opponent')

    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    df['Venue'] = 'Home'
    if 'GameID' not in df.columns:
        df['GameID'] = range(df.shape[0])

    c1 = df.filter(like='Team').columns
    c2 = c1.str.replace('Team', 'Opponent')
    
    swap = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    swap['Venue'] = 'Away'
    df['Result'] = np.where(df['Result'] == 'H', 'W', np.where(df['Result'] == 'A', 'L', df['Result']))
    swap['Result'] = np.where(swap['Result'] == 'H', 'L', np.where(swap['Result'] == 'A', 'W', swap['Result']))
    #swap['Result'] = np.where(swap['Result'] == 'H', 'A', np.where(swap['Result'] == 'A', 'H', 'D'))
    #print(swap)
    df = pd.concat([df,swap])
    df['Team'] = df['Team'].map(mapping)
    df['Opponent'] = df['Opponent'].map(mapping)
    #df['Result'] = np.where(df['Result'] != 'W', 'L', df['Result'])
    #df = df.rename(columns={'Team1':'Team', 'Team2':'Opponent'})
    df = df.sort_values(by='GameID').reset_index(drop=True)
    return df

In [9]:
def preprocess(total_df, source):
    total_df['TotalGoals'] = total_df['FTHG'] + total_df['FTAG']
    total_df['Date'] = pd.to_datetime(total_df['Date'], dayfirst=True)
    
    total_df = total_df.sort_values(by='Date')
    total_df['GameID'] = range(len(total_df))

    df = total_df
    #return df
    #return df.filter(like='>2.5')#.isna().sum()
    if source == 'csv':
        df = total_df[['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'FTHG', 'FTAG', 'FTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'BbAv>2.5', 'BbAv<2.5', 'AvgC>2.5', 'AvgC<2.5', 'B365H', 'B365D', 'B365A', 'AvgCH', 'AvgCD', 'AvgCA', 'B365>2.5', 'B365<2.5']]
        
        #df['O2.5'] = np.where(df['AvgC>2.5'].isna(),np.where(~df['BbAv>2.5'].isna(),df['BbAv>2.5'],df['B365>2.5']),df['AvgC>2.5'])
        #df['O2.5'] = df['B365>2.5']
        #df['U2.5'] = df['B365<2.5']
        df['O2.5'] = np.where(df['BbAv>2.5'].isna(),df['B365>2.5'],df['BbAv>2.5'])
        df['U2.5'] = np.where(df['AvgC<2.5'].isna(),np.where(~df['BbAv<2.5'].isna(),df['BbAv<2.5'],df['B365<2.5']),df['AvgC<2.5'])
        #df['U2.5'] = np.where(df['BbAv<2.5'].isna(),df['B365<2.5'],df['BbAv<2.5'])

        #['O2.5'].isna().sum())
        
        df['HomeTeamWinOdds'] = df['B365H']#np.where(df['AvgCH'].isna(),df['BbAvH'],df['AvgCH'])
        df['AwayTeamWinOdds'] = df['B365A']#np.where(df['AvgCA'].isna(),df['BbAvA'],df['AvgCA'])
        df['DrawOdds'] = df['B365D']#np.where(df['AvgCD'].isna(),df['BbAvD'],df['AvgCD'])
        
        df.columns = ['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'HomeTeamGoals', 'AwayTeamGoals', 'Result', 'Referee', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsonTarget', 'AwayTeamShotsonTarget', 'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamCorners', 'AwayTeamCorners', 'HomeTeamYellowCards', 'AwayTeamYellowCards', 'HomeTeamRedCards', 'AwayTeamRedCards', 'BbO2.5', 'BbU2.5', 'AvgO2.5', 'AvgU2.5','BbHome', 'BbDraw', 'BbAway', 'AvgH', 'AvgD', 'AvgA', 'B365O', 'B365U', 'O2.5', 'U2.5', 'HomeTeamWinOdds',  'AwayTeamWinOdds', 'DrawOdds']

        df = df[['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'HomeTeamGoals', 'AwayTeamGoals', 'Result', 'Referee', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsonTarget', 'AwayTeamShotsonTarget', 'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamCorners', 'AwayTeamCorners', 'HomeTeamYellowCards', 'AwayTeamYellowCards', 'HomeTeamRedCards', 'AwayTeamRedCards', 'O2.5', 'U2.5', 'HomeTeamWinOdds', 'DrawOdds', 'AwayTeamWinOdds']]
        df = df.sort_values(by='Date')

    '''
    c1 = df.filter(like='HomeTeam').columns

    c2 = c1.str.replace('HomeTeam', 'Team')
    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    c1 = df.filter(like='AwayTeam').columns

    c2 = c1.str.replace('AwayTeam', 'Opponent')

    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    df['Venue'] = 'Home'

    c1 = df.filter(like='Team').columns
    c2 = c1.str.replace('Team', 'Opponent')

    swap = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    swap['Venue'] = 'Away'
    #print(swap)
    df = pd.concat([df,swap])
    df['Team'] = df['Team'].map(mapping)
    df['Opponent'] = df['Opponent'].map(mapping)
    #df = df.rename(columns={'Team1':'Team', 'Team2':'Opponent'})
    df = df.sort_values(by='GameID').reset_index(drop=True)
    '''
    df = home_away(df)
    return df

In [10]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
map_values = {
    'Middlesboro': 'Middlesbrough',
    "Nott'm Forest": 'Forest',
    'Newcastle Utd': 'Newcastle',
    'Luton Town': 'Luton',
    'Manchester City': 'Man City',
    'Manchester Utd': 'Man United',
    'Sheffield Utd': 'Sheffield United',
    "Nott'ham Forest": 'Forest',
}
mapping = MissingDict(**map_values)

In [11]:
def create_predictors(df):
    df['VenueCode'] = df['Venue'].astype('category').cat.codes
    df['OpponentCode'] = df['Opponent'].astype('category').cat.codes
    df['Target'] = (df['TotalGoals'] > 2.5).astype('int')
    return df.reset_index(drop=True)

In [12]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values(by='Date')
    rolling_stats = group[cols].rolling(3, min_periods=2, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

def create_rolling(merged_df):
    cols = ['TotalGoals','TeamGoals', 'OpponentGoals', 'TeamShots', 'OpponentShots', 'TeamShotsonTarget', 'OpponentShotsonTarget', 'TeamCorners', 'OpponentCorners']
    new_cols = [f'{c}Rolling' for c in cols]
    df_rolling = merged_df.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols))
    df_rolling = df_rolling.droplevel('Team')
    df_rolling = df_rolling.sort_values('GameID')
    df_rolling.index = range(df_rolling.shape[0])
    return df_rolling

In [13]:
def calculate_difs(df, cols):
    new_cols = [f'{c}Rolling_Dif' for c in cols]
    #print(new_cols)
    for i,c in enumerate(cols):
        df[new_cols[i]] = df[f'Team{c}Rolling'] - df[f'Opponent{c}Rolling']
    return df, new_cols

In [14]:
'''
print('getting prem')
prem_df = #get_prem_data()
print('getting schedule')
sched23 = get_current_season()
#sched23 = get_schedule()

new_df2023 = pd.concat([prem_df, sched23])
new_df2023.to_csv('cache/total_df.csv')
print(new_df2023.columns)
print('preprocess')
df1 = preprocess(new_df2023, 'csv')

print('create predictors')
full_df = create_predictors(df1)
print('getting elo')
merged_df = get_elo_data(full_df)
print('create rolling')
df_rolling = create_rolling(merged_df)
print('predicting')
'''
df_rolling = pd.read_csv('/Users/Gautham/Projects/Betting/cache/rolling.csv')
curr = pd.read_csv('/Users/Gautham/Projects/Betting/cache/current_season.csv')
curr[['BbAv>2.5', 'BbAv<2.5', 'BbAvH', 'BbAvD', 'BbAvA']] = None

In [15]:
df_rolling['WinTarget'] = df_rolling['Result'].astype('category').cat.codes#df_rolling['Result'].astype('category').cat.codes#(df_rolling['Result'] == 'W').astype('int')#np.where(((df_rolling['Result'] == 'H') & (df_rolling['VenueCode'] == 1)) | ((df_rolling['Result'] == 'A') & (df_rolling['VenueCode'] == 0)),  1, 0).astype('int')#df_rolling['Result'].astype('category').cat.codes


In [16]:
numeric_cols = ['Shots', 'ShotsonTarget', 'Fouls', 'Corners', 'YellowCards', 'RedCards']
cols = ['Goals','Shots', 'ShotsonTarget', 'Corners']
df_rolling, new_cols = calculate_difs(df_rolling, cols)


In [193]:
df_rolling.to_csv('cache/rolling_dif.csv')

In [101]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
    

def random_forest_win(data, predictors, train, test, call_type, target_col, model_type):  
    if model_type == 'RF':
        model = RandomForestClassifier(n_estimators=50, min_samples_split=7, random_state=1) 
    else: 
        params = {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'reg_lambda': 0, 'subsample': 0.8}
        model = xgb.XGBClassifier(**params)#, objective='binary:logistic')

    model = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=17, min_samples_split=5, n_estimators=100)

     
    '''
    param_grid = {
	    "max_depth": [3],
	    "learning_rate": [0.01],
	    "gamma": [0, 0.25, 1],
	    "reg_lambda": [0, 1, 10],
	    "scale_pos_weight": [1, 3, 5],
	    "subsample": [0.8],
	    "colsample_bytree": [0.5],
	}
    '''
    param_grid = {
	    "max_depth": [3],
	    "learning_rate": [0.01],
	    "gamma": [0, 0.25, 1],
	    "reg_lambda": [0, 1, 10],
	    "subsample": [0.8],
	    "colsample_bytree": [0.5],
	}

    params = {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'reg_lambda': 10, 'scale_pos_weight': 1, 'subsample': 0.8}
    params = {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'reg_lambda': 0, 'subsample': 0.8}
    #params = {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'reg_lambda': 10, 'subsample': 0.8}
    
    #model = xgb.XGBClassifier(**params)#, objective='binary:logistic')
    #model = xgb.XGBRegressor()
    #grid_cv = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, scoring='roc_auc')
    #_ = grid_cv.fit(train[predictors], train[target_col])
    #print(grid_cv.best_score_, grid_cv.best_params_)

    #model.fit(train[predictors], train['Target'])
    model.fit(train[predictors], train[target_col])
    preds = model.predict(test[predictors])
    #return preds,1,1
    #if call_type == 'predict':
    #    return preds, model
    #combined = pd.DataFrame(dict(actual=test['Target'], prediction=preds), index = test.index)
    #print(test.groupby('GameID').filter(lambda x: x['prediction'].sum()==1))
    precision = precision_score(test[target_col], preds, average='micro')
    accuracy = accuracy_score(test[target_col], preds)
    combined = pd.DataFrame(dict(actual=test[target_col], prediction=preds), index = test.index)
    c1 = combined.merge(data[['Date', 'Team', 'Opponent', 'TeamGoals', 'OpponentGoals','GameID', 'Venue']], left_index=True, right_index=True)

    # predictions where both home and away team prediction match
    predictions = c1.groupby('GameID').filter(lambda x: x['prediction'].sum()==1)
    #print(predictions)
    table = c1.sort_index()[['GameID','Date','Team', 'Opponent', 'prediction', 'actual']].merge(c1[['GameID','Date','Team', 'Opponent', 'prediction']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
   
    table = table[table['prediction_x'] == table['prediction_y']]#.filter(lambda x: x['prediction_x'] != x['prediction_y'])
    precision1 = precision_score(table['actual'], table['prediction_x'], average='micro')
    accuracy1 = accuracy_score(table['actual'], table['prediction_x'])
    #print(precision1, accuracy1, precision2, accuracy2)
    #precision, accuracy = 1,1
    #precision = cross_val_score(model, train[predictors], train['TotalGoals'], scoring='neg_mean_absolute_error')
    return table, precision1, accuracy1, model, combined

In [102]:
predictors = ['VenueCode', 'OpponentCode', 'ELODif']
predictors = ['VenueCode', 'ELODif']
new_cols = ['TotalGoalsRolling',
 'TeamGoalsRolling',
 'OpponentGoalsRolling',
 'TeamShotsRolling',
 'OpponentShotsRolling',
 'TeamShotsonTargetRolling',
 'OpponentShotsonTargetRolling',
 'TeamCornersRolling',
  'OpponentCornersRolling']
cols = ['TotalGoalsRolling','GoalsRolling_Dif', 'ShotsRolling_Dif', 'ShotsonTargetRolling_Dif', 'CornersRolling_Dif']


#predictors = ['VenueCode', 'ELODif']
new_df = df_rolling.dropna(subset=['O2.5'])
new_df.shape
train = new_df[new_df['Date'] < '2016-08']
test = new_df[new_df['Date'] >= '2016-08']# and new_df['Date'] < '2022-colst = new_df.loc[(new_df['Date'] >= '2016-08')]# & (new_df['Date'] < '2022-05')]
table, precision, accuracy, model, combined = random_forest_win(df_rolling, predictors+cols, train, test, 'model', 'Target', 'XGB')
precision, accuracy, table

(0.5391061452513967,
 0.5391061452513967,
       GameID_x        Date       Team_x        Opponent_x  prediction_x  \
 6         6083  2016-08-13      Watford       Southampton             1   
 8         6084  2016-08-13         Hull         Leicester             1   
 10        6085  2016-08-13        Stoke     Middlesbrough             0   
 12        6086  2016-08-13     Man City        Sunderland             1   
 14        6087  2016-08-14    Liverpool           Arsenal             1   
 ...        ...         ...          ...               ...           ...   
 4622      8442  2023-09-16    Newcastle         Brentford             0   
 4624      8443  2023-09-16    Tottenham  Sheffield United             1   
 4626      8444  2023-09-16     West Ham          Man City             0   
 4630      8446  2023-09-17      Everton           Arsenal             0   
 4632      8447  2023-09-17  Bournemouth           Chelsea             0   
 
       actual  GameID_y            Team_y   

In [90]:
c1 = combined.merge(df_rolling[['Date', 'Team', 'Opponent', 'TeamGoals', 'OpponentGoals','GameID', 'Venue']], left_index=True, right_index=True)
combined
# predictions where both home and away team prediction match
predictions = c1.groupby('GameID').filter(lambda x: x['prediction'].sum()==1)
#predictions = c1.groupby('GameID')[['Team', 'Opponent','actual', 'prediction']].sum()
#predictions[['actual','prediction']] = predictions[['actual','prediction']] / 2 .filter(lambda x: x['prediction'].sum()!=1)#[['GameID','Date', 'prediction','actual']]
#predictions = predictions[predictions['Date'] > '2023-08']

pred_with_odds = predictions.merge(df_rolling[['GameID','O2.5', 'U2.5']], how='inner',on='GameID').drop_duplicates()

#pred_with_odds = pred_with_odds[['GameID', 'O2.5', 'U2.5', 'prediction', 'actual']].drop_duplicates()
#pred_with_odds[pred_with_odds['actual'] == pred_with_odds['prediction']].shape[0] / pred_with_odds.shape[0] 
merged = c1.merge(c1, left_on=['GameID','Date', 'Team'], right_on=['GameID','Date', 'Opponent'])
predictions
c1
pred_with_odds
#merged[((merged['prediction_x'] == 1) & (merged['prediction_y'] == 0)) | ((merged['prediction_x'] == 0) & (merged['prediction_y'] == 1))]['actual_x'].value_counts()

,actual,prediction,Date,Team,Opponent,TeamGoals,OpponentGoals,GameID,Venue,O2.5,U2.5
0,0,1,2016-08-13,Swansea,Burnley,1,0,6080,Away,2.30,1.61
2,0,0,2016-08-13,Burnley,Swansea,0,1,6080,Home,2.30,1.61
4,0,0,2016-08-13,Crystal Palace,West Brom,0,1,6082,Home,2.50,1.52
6,0,1,2016-08-13,West Brom,Crystal Palace,1,0,6082,Away,2.50,1.52
8,0,0,2016-08-13,Watford,Southampton,1,1,6083,Away,2.06,1.75
...,...,...,...,...,...,...,...,...,...,...,...
3692,1,1,2023-09-16,Brighton,Man United,3,1,8441,Away,1.44,2.88
3694,1,1,2023-09-16,Wolves,Liverpool,1,3,8445,Home,1.44,2.56
3696,1,0,2023-09-16,Liverpool,Wolves,3,1,8445,Away,1.44,2.56
3698,0,1,2023-09-17,Bournemouth,Chelsea,0,0,8447,Home,1.67,2.19


In [175]:
610/(610+522)

0.5388692579505301

In [192]:
def prep_data(week):
    rolling = pd.read_csv('cache/rolling.csv')
    elo = pd.read_csv('cache/elo.csv')
    schedule = pd.read_csv('cache/schedule.csv')
    #print(rolling['Date'])
    recent_rolling = rolling[rolling['Date'] >= '2023-09-16']
    #schedule.loc[(schedule['Date'] == '23/09/23') | (schedule['Date'] == '24/09/23')]
    #print(recent_rolling)
    sched = pd.DataFrame()
    #for date in dates:
    sched = pd.concat([sched, schedule.loc[schedule['week'] >= week]])#(schedule['Date'] == date)]])
   
    sched['Result'] = -1
    sched = home_away(sched)
    
    #print(sched)
    sched = sched.merge(recent_rolling[['Team', 'TeamELO']], on='Team', how='inner')
    sched = sched.merge(recent_rolling[['Opponent', 'OpponentELO']], on='Opponent', how='inner')
    sched['ELODif'] = sched['TeamELO'] - sched['OpponentELO']
    sched['Date'] = pd.to_datetime(sched['Date'], dayfirst=True).dt.date

    new = pd.concat([elo, sched[['Date', 'Team', 'Opponent', 'TeamELO', 'OpponentELO', 'ELODif', 'Venue', "GameID"]]])
    #print(new.columns)
    #return new
    new['Date'] = pd.to_datetime(new['Date'], dayfirst=True).dt.date
    print(elo.Date.dtype, sched.Date.dtype)

    new = create_predictors(new)
    new = create_rolling(new)
    numeric_cols = ['Shots', 'ShotsonTarget', 'Fouls', 'Corners', 'YellowCards', 'RedCards']
    cols = ['Goals','Shots', 'ShotsonTarget', 'Corners']
    #print(new.columns, 'h')
    new, dif_cols = calculate_difs(new, cols)

    new['WinLoseTarget'] = new['Result'].astype('category').cat.codes #(new['Result'] == 'W').astype('int')
    new['WinTarget'] = (new['Result'] == 'W')#new['Result'].astype('category').cat.codes
    new['DrawTarget'] = (new['Result'] == 'D')
    new['LossTarget'] = (new['Result'] == 'L')
    new.to_csv('new.csv')
    return dif_cols

def predict(date, pred_type, dif_cols):
    #new['WinTarget'] = codes.cat.codes
    #cats = codes.cat.categories
    #print(new['WinTarget'], new['Result'])
    #return new, 'hi'
    new = pd.read_csv('new.csv')
    predictors = ['VenueCode', 'OpponentCode', 'ELODif'] + dif_cols
    #predictors = ['VenueCode', 'ELODif']
    #print(new)
    new_df = new.dropna(subset=['O2.5'])
    train = new_df[new_df['Date'] < '2018-09-18']
    #return train
    #test = new_df[new_df['Date'] >= '2022-08' and new_df['Date'] < '2022-08']
    #new_df.Date
    test = new_df[(new_df['Date'] >= '2018-08') & (new_df['Date'] < '2023-08')]
    pred = new.loc[(new['Date'] >= date)]
    #pred_type = 'WinTarget'
    table, prec, acc, model, combined = random_forest_win(df_rolling, predictors, train, test, 'predict', pred_type, 'RF')
    #print(combined)
    preds = model.predict(pred[predictors])
    #c1 = preds.merge(test[['Date', 'Team', 'Opponent', 'TotalGoals', 'GameID']], left_index=True, right_index=True)
    
    return preds, table, pred


#table[table[('preds_x' == 'preds_y')]]


In [193]:
date = '2023-09-25'
dif_cols = prep_data(7)#['30/09/23', '01/10/23', '02/10/23'])
preds_wl, combined1, c1 = predict(date, 'WinLoseTarget', dif_cols)
preds_ou, combined2, c2 = predict(date, 'Target', dif_cols)
preds_w, combined3, c3 = predict(date, 'WinTarget', dif_cols)
preds_d, combined4, c4 = predict(date, 'DrawTarget', dif_cols)
preds_l, combined5, c5 = predict(date, 'LossTarget', dif_cols)

object object


In [194]:
c1['preds'] = preds_wl
table_wl = c1.sort_index()[['GameID','Date','Team', 'Opponent', 'preds']].merge(c1[['GameID','Date','Team', 'Opponent', 'preds']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
#c1
table_wl


,GameID_x,Date,Team_x,Opponent_x,preds_x,GameID_y,Team_y,Opponent_y,preds_y
0,0,2023-09-30,Aston Villa,Brighton,2,0,Brighton,Aston Villa,1
2,1,2023-09-30,Arsenal,Bournemouth,2,1,Bournemouth,Arsenal,1
4,2,2023-09-30,Everton,Luton,2,2,Luton,Everton,1
6,3,2023-09-30,Crystal Palace,Man United,1,3,Man United,Crystal Palace,2
8,5,2023-09-30,Tottenham,Liverpool,2,5,Liverpool,Tottenham,2
10,6,2023-09-30,West Ham,Sheffield United,2,6,Sheffield United,West Ham,1
12,7,2023-09-30,Wolves,Man City,1,7,Man City,Wolves,2
14,9,2023-10-02,Chelsea,Fulham,1,9,Fulham,Chelsea,2
16,12,2023-10-07,Bournemouth,Everton,1,12,Everton,Bournemouth,2
18,13,2023-10-07,Sheffield United,Fulham,1,13,Fulham,Sheffield United,2


In [197]:
#preds

c2['preds'] = preds_ou
table_ou = c2.sort_index()[['GameID','Date','Team', 'Opponent', 'preds']].merge(c2[['GameID','Date','Team', 'Opponent', 'preds']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
table_ou[table_ou['preds_x'] == table_ou['preds_y']]

,GameID_x,Date,Team_x,Opponent_x,preds_x,GameID_y,Team_y,Opponent_y,preds_y
0,0,2023-09-30,Aston Villa,Brighton,0,0,Brighton,Aston Villa,0
4,2,2023-09-30,Everton,Luton,0,2,Luton,Everton,0
8,5,2023-09-30,Tottenham,Liverpool,1,5,Liverpool,Tottenham,1
12,7,2023-09-30,Wolves,Man City,1,7,Man City,Wolves,1
20,14,2023-10-07,Tottenham,Luton,1,14,Luton,Tottenham,1
22,15,2023-10-07,Man United,Brentford,1,15,Brentford,Man United,1
24,16,2023-10-08,Arsenal,Man City,1,16,Man City,Arsenal,1


In [198]:
c3['preds'] = preds_w
table_w = c3.sort_index()[['GameID','Date','Team', 'Opponent', 'preds']].merge(c3[['GameID','Date','Team', 'Opponent', 'preds']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
table_w

,GameID_x,Date,Team_x,Opponent_x,preds_x,GameID_y,Team_y,Opponent_y,preds_y
0,0,2023-09-30,Aston Villa,Brighton,False,0,Brighton,Aston Villa,False
2,1,2023-09-30,Arsenal,Bournemouth,True,1,Bournemouth,Arsenal,False
4,2,2023-09-30,Everton,Luton,True,2,Luton,Everton,False
6,3,2023-09-30,Crystal Palace,Man United,False,3,Man United,Crystal Palace,True
8,5,2023-09-30,Tottenham,Liverpool,False,5,Liverpool,Tottenham,False
10,6,2023-09-30,West Ham,Sheffield United,True,6,Sheffield United,West Ham,False
12,7,2023-09-30,Wolves,Man City,False,7,Man City,Wolves,True
14,9,2023-10-02,Chelsea,Fulham,False,9,Fulham,Chelsea,False
16,12,2023-10-07,Bournemouth,Everton,False,12,Everton,Bournemouth,False
18,13,2023-10-07,Sheffield United,Fulham,False,13,Fulham,Sheffield United,True


In [ ]:
c5['preds'] = preds_l
table_l = c5.sort_index()[['GameID','Date','Team', 'Opponent', 'preds']].merge(c5[['GameID','Date','Team', 'Opponent', 'preds']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
table_l

In [142]:
c4['preds'] = preds_d
table_d = c4.sort_index()[['GameID','Date','Team', 'Opponent', 'preds']].merge(c4[['GameID','Date','Team', 'Opponent', 'preds']], left_on=['Date', 'Team'], right_on=['Date', 'Opponent']).drop_duplicates(['GameID_x'])
table_d

,GameID_x,Date,Team_x,Opponent_x,preds_x,GameID_y,Team_y,Opponent_y,preds_y
0,0,23/09/23,Brentford,Everton,False,0,Everton,Brentford,False
2,2,23/09/23,Fulham,Crystal Palace,False,2,Crystal Palace,Fulham,False
4,3,23/09/23,Luton,Wolves,False,3,Wolves,Luton,False
6,5,24/09/23,Tottenham,Arsenal,False,5,Arsenal,Tottenham,False
8,6,24/09/23,Brighton,Bournemouth,False,6,Bournemouth,Brighton,False
10,7,24/09/23,Aston Villa,Chelsea,False,7,Chelsea,Aston Villa,False
12,8,24/09/23,Liverpool,West Ham,False,8,West Ham,Liverpool,False
14,9,24/09/23,Sheffield United,Newcastle,False,9,Newcastle,Sheffield United,False


In [118]:
table_ou.merge(table_wl, on='GameID_x').drop(columns=['Date_y', 'Team_x_y', 'Opponent_x_y'])

,GameID_x,Date_x,Team_x_x,Opponent_x_x,preds_x_x,GameID_y_x,Team_y_x,Opponent_y_x,preds_y_x,preds_x_y,GameID_y_y,Team_y_y,Opponent_y_y,preds_y_y
0,0,23/09/23,Brentford,Everton,2,0,Everton,Brentford,1,0,0,Everton,Brentford,0
1,2,23/09/23,Fulham,Crystal Palace,1,2,Crystal Palace,Fulham,2,0,2,Crystal Palace,Fulham,0
2,3,23/09/23,Luton,Wolves,1,3,Wolves,Luton,1,0,3,Wolves,Luton,1
3,5,24/09/23,Tottenham,Arsenal,2,5,Arsenal,Tottenham,2,1,5,Arsenal,Tottenham,1
4,6,24/09/23,Brighton,Bournemouth,2,6,Bournemouth,Brighton,1,1,6,Bournemouth,Brighton,1
5,7,24/09/23,Aston Villa,Chelsea,1,7,Chelsea,Aston Villa,2,1,7,Chelsea,Aston Villa,1
6,8,24/09/23,Liverpool,West Ham,2,8,West Ham,Liverpool,1,1,8,West Ham,Liverpool,1
7,9,24/09/23,Sheffield United,Newcastle,1,9,Newcastle,Sheffield United,2,1,9,Newcastle,Sheffield United,1
